In [13]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import rayleigh

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation

import sys

In [ ]:
sys.path.append("/home/michael/SBDPP_SIM") # your path to the SBDPP_sim folder
import simulation

In [ ]:
L = 50.0
M = 4

r_values = np.arange(0, 5.01, 0.01)

def density(r):
    return (1 / (2 * np.pi)) * np.exp(-0.5 * r**2)

radial_density_values = density(r_values)
q_values = np.arange(0, 1.0, 0.001)
inverse_radial_values = rayleigh.ppf(q_values, scale = 1.0)

g2 = simulation.PyGrid2(
    M = M,
    areaLen = [L, L],
    cellCount = [L, L],
    isPeriodic = True,
    birthRates = [1] * M,
    deathRates = [0] * M,
    ddMatrix = [0.1, 0.3, 0.3, 0.3,
                0.3, 0.1, 0.3, 0.3,
                0.3, 0.3, 0.1, 0.3,
                0.3, 0.3, 0.3, 0.1],
    birthX = [q_values.tolist()] * M,
    birthY = [inverse_radial_values.tolist()] * M,
    deathX = [[r_values.tolist()]*M] * M,
    deathY = [[radial_density_values.tolist()] * M] * M,
    cutoffs = [5.0] * M * M,
    seed = 42,
    rtimeLimit = 7200.0
)


N = 1000

coordinates = []
for _ in range(M):
    group = [[np.random.uniform(0, L), np.random.uniform(0, L)] for _ in range(N)]
    coordinates.append(group)

g2.placePopulation(coordinates)

print("Birth rate:", g2.total_birth_rate)
print("Death rate:", g2.total_death_rate)

Birth rate: 4000.0
Death rate: 1595.1301791971398


In [16]:
data = list()

for t in (pbar := tqdm(range(10))):
    for c in range(g2.get_num_cells()):
        for n in range(M):
            coords = g2.get_cell_coords(c, n)
            for i in range(len(coords)):
                x,y = coords[i]
                data.append([t, x, y, n])
    g2.run_for(1)    
    pbar.set_description(f"Population: {g2.total_population}, Progress")

Population: 13609, Progress: 100%|██████████| 10/10 [00:10<00:00,  1.02s/it]


In [17]:
df = pd.DataFrame(columns = ['time', 'x', 'y', 's'], data = data)
display(df.set_index('time'))

,x,y,s
time,,,
0,0.530362,0.231104,3
0,1.718885,0.159825,0
0,1.212915,0.529530,1
0,1.275332,0.845118,2
0,2.128303,0.572695,1
...,...,...,...
9,48.149622,49.100466,0
9,48.277174,49.806216,0
9,49.760074,49.472692,0


In [18]:
movie_writer = animation.FFMpegWriter(
    fps = 2,
    codec = 'h264',
    extra_args = ['-vcodec', 'libx264']
)

In [ ]:
matplotlib.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg' # Set the path to ffmpeg

df = df.sort_values('time')
output_file = "points_animation.mp4"

fig, ax = plt.subplots(figsize=(8, 8))

ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.set_xlabel('x')
ax.set_ylabel('y')
sc = ax.scatter([], [], c = [], cmap = 'plasma', s = 20, alpha = 0.6)

def update(frame):
    current_df = df[df['time'] == frame]
    sc.set_offsets(current_df[['x', 'y']].values)
    sc.set_array(current_df['s'].values)
    ax.set_title(f"Time: {frame}")
    return sc,

ani = FuncAnimation(fig, update, frames=df['time'].nunique(), blit = True)

ani.save(output_file, writer = movie_writer)
print(f"Animation saved as {output_file}")
plt.close(fig)

Animation saved as points_animation.mp4
